#### Libraries

In [109]:
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import string
import gensim
import matplotlib.pyplot as plt

from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from nltk import pos_tag, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import SnowballStemmer
from nltk.stem.snowball import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim import corpora, models
from nltk.stem.porter import *

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Busss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Busss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Loading the data

In [18]:
df = pd.read_csv('data/data.csv')
df.shape

C:\Users\Busss\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(204135, 12)

In [19]:
df.head()

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN


#### Removing all columns except the 'content' column

In [20]:
remove = [i for i in df.columns if i!='content']

In [21]:
df.drop(remove, axis = 1, inplace=True)
df.head()

,content
0,And never more so than in Showtime’s new...
1,AlphaGo’s victory isn’t a defeat for hum...
2,Super Deluxe built a weird internet empi...
3,Steven Yang quit his job at Google in th...
4,Ahead of Black Panther’s 2018 theatrical...


In [22]:
df.shape

(204135, 1)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204135 entries, 0 to 204134
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  188732 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [24]:
df.dropna(inplace=True)

In [25]:
df.shape

(188732, 1)

#### Text preprocess functions

In [26]:
stemmer = SnowballStemmer("english")

def lem_stem(word):
    return stemmer.stem(WordNetLemmatizer().lemmatize(word))

def preprocess(text):
    return [lem_stem(token) for token in gensim.utils.simple_preprocess(text) if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]            


#### Checking the functions at work

In [27]:
print('original:',df['content'].iloc[2][:100])
print("after:",preprocess(df['content'].iloc[2][:100]))

original:       Super Deluxe built a weird internet empire. Can it succeed on TV? When Wolfgang Hammer talks a
after: ['super', 'delux', 'built', 'weird', 'internet', 'empir', 'succeed', 'wolfgang', 'hammer', 'talk']


#### Preprocessing the data

In [28]:
pp_data = df['content'].map(preprocess)

In [29]:
pp_data[:10]

0    [showtim, seri, reviv, spoiler, ahead, episod,...
1    [alphago, victori, defeat, human, opportun, lo...
2    [super, delux, built, weird, internet, empir, ...
3    [steven, yang, quit, googl, summer, build, pro...
4    [ahead, black, panther, theatric, releas, marv...
5    [facebook, instant, articl, promis, transform,...
6    [weapon, weapon, year, artist, technolog, enth...
7    [insid, busi, imagin, futur, dubai, world, loo...
8    [legal, threat, disgruntl, client, insid, uber...
9    [genius, quiet, laid, bunch, engin, surviv, me...
Name: content, dtype: object

#### Bag-of-Words

In [30]:
dict_of_words = gensim.corpora.Dictionary(pp_data)

In [182]:
for i, (key, value) in enumerate(dict_of_words.iteritems()):
    print(key, value)
    if i == 10:
        break

0 abandon
1 abil
2 absenc
3 acknowledg
4 act
5 adventur
6 age
7 agent
8 ahead
9 alarm
10 alien


#### Total number of words in dict:

In [184]:
print('total:',len(dict_of_words.iteritems()))

total: 42806


#### Filter out tokens
remove < 'no_below', 
remove > 'no_above',
then keep only the first 'keep_n' most

In [34]:
dict_of_words.filter_extremes(no_below=15, no_above=0.1, keep_n=50000)

In [35]:
bow_for_docs = [dict_of_words.doc2bow(i) for i in pp_data]

#### How many words in the content at the i-th location
(i==2 in this case)

In [36]:
len(bow_for_docs[2])

533

#### Check the words at that location
(id, count)

##### Ordered by how they appear in the content - top 10

In [44]:
bow_for_docs[2][:10]

[(5, 1),
 (11, 1),
 (16, 1),
 (18, 4),
 (26, 1),
 (31, 1),
 (39, 3),
 (40, 3),
 (41, 1),
 (43, 1)]

##### Ordered by the most frequent - top 10

In [49]:
sorted(bow_for_docs[2],  key=lambda x: x[1], reverse=True)[:10]

[(517, 41),
 (649, 40),
 (728, 20),
 (980, 13),
 (770, 11),
 (638, 10),
 (686, 9),
 (759, 8),
 (913, 8),
 (931, 8)]

##### Ordered by the least frequent - top 10

In [51]:
sorted(bow_for_docs[2],  key=lambda x: x[1])[:10]

[(5, 1),
 (11, 1),
 (16, 1),
 (26, 1),
 (31, 1),
 (41, 1),
 (43, 1),
 (50, 1),
 (61, 1),
 (66, 1)]

#### Check the actual words

In [180]:
limit = 10
space = '    '
tmp_words = bow_for_docs[2]

print(f'LOC | WORD NUMBER |  WORD FREQ  | THE ACTUAL WORD')
print('--------------------------------------------------')
for i in range(len(tmp_words)):
    num = str(tmp_words[i][0])
    while len(num) < 3: num = ' '+num
        
    freq = str(tmp_words[i][1])
    while len(freq) < 3: freq = ' '+freq

    loc = str(i)
    while len(loc) < 3: loc = ' '+loc

    print(f'{loc} | {space}{num}{space} | {space}{freq}{space} | {space}{dict_of_words[tmp_words[i][0]]}')
    if i == limit: break

LOC | WORD NUMBER |  WORD FREQ  | THE ACTUAL WORD
--------------------------------------------------
  0 |       5     |       1     |     adventur
  1 |      11     |       1     |     ambiti
  2 |      16     |       1     |     arc
  3 |      18     |       4     |     audienc
  4 |      26     |       1     |     bizarr
  5 |      31     |       1     |     board
  6 |      39     |       3     |     budget
  7 |      40     |       3     |     built
  8 |      41     |       1     |     cabl
  9 |      43     |       1     |     cancel
 10 |      49     |       5     |     charact


### TF-IDF

In [105]:
idf = models.TfidfModel(bow_for_docs)

##### Transforming

In [108]:
tdf_data = idf[bow_for_docs]

##### (id, score)

In [118]:
for i in tdf_data:
    print(i)
    break

[(0, 0.015478959057446923), (1, 0.012365447563896025), (2, 0.01890746695108075), (3, 0.012502690603094915), (4, 0.011984062827162469), (5, 0.020523541608644492), (6, 0.015227182589769604), (7, 0.11081895116417972), (8, 0.011921362809765527), (9, 0.01665278701142758), (10, 0.03410854322226277), (11, 0.018483863961796826), (12, 0.016047319220607432), (13, 0.034013609410010866), (14, 0.012789150604182389), (15, 0.016250003948498216), (16, 0.0316115763113638), (17, 0.016496878996472127), (18, 0.18158696326799126), (19, 0.027367336605085904), (20, 0.019546777774875762), (21, 0.0742679533618682), (22, 0.031015388537486425), (23, 0.026869514749103846), (24, 0.021000730869090756), (25, 0.03046593774327872), (26, 0.02038032475300019), (27, 0.013288816494756946), (28, 0.022541512871759723), (29, 0.021959038197401203), (30, 0.018645903434227902), (31, 0.011829266273268656), (32, 0.04139491682670043), (33, 0.01984032567888331), (34, 0.016582602982426465), (35, 0.026315185252322112), (36, 0.0206505

### LDA
Training

In [131]:
lda = gensim.models.LdaMulticore(bow_for_docs, num_topics=10, id2word=dict_of_words, passes=3, workers=3)

##### Lets view the words

In [132]:
for i, topic in lda.print_topics(-1):
    print(f"Topic: {i} \nWords: {topic}\n---\n")

Topic: 0 
Words: 0.011*"reuter" + 0.009*"price" + 0.008*"bank" + 0.007*"stock" + 0.007*"insur" + 0.006*"delay" + 0.006*"economi" + 0.006*"worker" + 0.006*"thomson" + 0.005*"financ"
---

Topic: 1 
Words: 0.015*"player" + 0.013*"season" + 0.009*"sport" + 0.008*"leagu" + 0.008*"coach" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball" + 0.004*"athlet" + 0.004*"match"
---

Topic: 2 
Words: 0.008*"russia" + 0.008*"russian" + 0.008*"korea" + 0.007*"islam" + 0.007*"china" + 0.006*"syria" + 0.005*"minist" + 0.005*"nuclear" + 0.005*"muslim" + 0.005*"iran"
---

Topic: 3 
Words: 0.005*"sander" + 0.005*"poll" + 0.004*"nomine" + 0.004*"cruz" + 0.003*"primari" + 0.003*"nomin" + 0.003*"bush" + 0.003*"tweet" + 0.003*"liber" + 0.003*"convent"
---

Topic: 4 
Words: 0.005*"film" + 0.004*"music" + 0.004*"movi" + 0.003*"charact" + 0.003*"artist" + 0.002*"song" + 0.002*"beauti" + 0.002*"girl" + 0.002*"father" + 0.002*"writer"
---

Topic: 5 
Words: 0.007*"crime" + 0.007*"attorney" + 0.006*"lawyer" + 0.006*"sho

#### Topic Classification

In [129]:
'Topic: 0 Words: 0.015*"player" + 0.012*"season" + 0.009*"sport" + 0.008*"leagu" + 0.007*"coach" + 0.007*"olymp" + 0.006*"athlet" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball"'

'Topic: 0 Words: 0.015*"player" + 0.012*"season" + 0.009*"sport" + 0.008*"leagu" + 0.007*"coach" + 0.007*"olymp" + 0.006*"athlet" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball"'

### LDA + TF-IDF
Training

In [187]:
lda_idf = gensim.models.LdaMulticore(tdf_data, num_topics=5, id2word=dict_of_words, passes=4, workers=6)

In [188]:
for i, topic in lda_idf.print_topics(-1):
    print(f"Topic: {i} \nWords: {topic}\n---\n")

Topic: 0 
Words: 0.004*"sander" + 0.003*"cruz" + 0.003*"comey" + 0.003*"poll" + 0.003*"russian" + 0.003*"nomine" + 0.003*"russia" + 0.002*"intellig" + 0.002*"ryan" + 0.002*"rubio"
---

Topic: 1 
Words: 0.005*"season" + 0.005*"player" + 0.004*"leagu" + 0.004*"coach" + 0.003*"storm" + 0.003*"yanke" + 0.003*"hurrican" + 0.003*"water" + 0.002*"footbal" + 0.002*"basebal"
---

Topic: 2 
Words: 0.002*"film" + 0.002*"music" + 0.001*"movi" + 0.001*"food" + 0.001*"artist" + 0.001*"song" + 0.001*"charact" + 0.001*"space" + 0.001*"season" + 0.001*"anim"
---

Topic: 3 
Words: 0.005*"reuter" + 0.003*"china" + 0.003*"korea" + 0.003*"thomson" + 0.002*"bank" + 0.002*"delay" + 0.002*"syria" + 0.002*"stock" + 0.002*"price" + 0.002*"nuclear"
---

Topic: 4 
Words: 0.003*"student" + 0.002*"protest" + 0.002*"sexual" + 0.002*"facebook" + 0.002*"shoot" + 0.002*"crime" + 0.002*"educ" + 0.002*"violenc" + 0.002*"judg" + 0.002*"victim"
---



#### Topic Classification

In [189]:
'Topic: 0 Words: 0.010*"olymp" + 0.009*"yanke" + 0.008*"player" + 0.007*"athlet" + 0.007*"met" + 0.006*"cuban" + 0.006*"cuba" + 0.006*"basebal" + 0.006*"leagu" + 0.006*"season"'

'Topic: 0 Words: 0.010*"olymp" + 0.009*"yanke" + 0.008*"player" + 0.007*"athlet" + 0.007*"met" + 0.006*"cuban" + 0.006*"cuba" + 0.006*"basebal" + 0.006*"leagu" + 0.006*"season"'

### Testing
###### Using these parameters

In [190]:
loc = 4321
tmp_doc = pp_data[loc]
print(tmp_doc)

['dodger', 'reliev', 'paco', 'rodriguez', 'fire', 'pitch', 'loui', 'cardin', 'season', 'near', 'perfect', 'think', 'mean', 'spring', 'train', 'know', 'like', 'exhibit', 'season', 'litter', 'spotless', 'era', 'high', 'bat', 'averag', 'sent', 'minor', 'dodger', 'plenti', 'roster', 'open', 'uncertain', 'bullpen', 'think', 'left', 'hander', 'paco', 'rodriguez', 'earn', 'spot', 'like', 'think', 'excel', 'outing', 'dodger', 'loss', 'giant', 'friday', 'night', 'camelback', 'ranch', 'rodriguez', 'perfect', 'spring', 'reason', 'demand', 'appear', 'rodriguez', 'given', 'inning', 'surrend', 'hit', 'walk', 'strike', 'reason', 'slam', 'dunk', 'team', 'option', 'good', 'reason', 'pitch', 'came', 'friday', 'game', 'replac', 'zack', 'greink', 'out', 'fourth', 'inning', 'runner', 'second', 'base', 'right', 'hander', 'given', 'run', 'giant', 'lead', 'situat', 'unlik', 'dodger', 'open', 'camp', 'surplus', 'left', 'hand', 'reliev', 'spent', 'season', 'howel', 'hard', 'come', 'day', 'remain', 'season', 'op

###### Using this article

In [191]:
print('original:',df['content'].iloc[loc])

original: Dodgers reliever Paco Rodriguez fires a pitch against the St. Louis Cardinals last season.', 'If you’re nearly perfect, you’d think that would mean something in spring training. You know, like a job. But the exhibition season is littered with those with spotless ERAs and high batting averages sent back to the minors.', 'The Dodgers have plenty of roster openings in their uncertain bullpen and you’d have to think left-hander Paco Rodriguez has earned a spot. Anyway, you’d like to think so.', 'After another excellent outing in the Dodgers’ 8-4 loss to the Giants on Friday night at Camelback Ranch, Rodriguez has been about as perfect this spring as anyone would have reason to demand.', 'In his eight appearances, Rodriguez has not given up a run. In his 8 1/3 innings, he has surrendered only three hits and no walks while striking out 10. The only reason he’s not a slam dunk to make the team is because he can still be optioned. And that’s not a good enough reason, not the way he’s

#### LDA model

##### Lets check which topic 'tmp_doc' belongs to

In [192]:
for i, conf in sorted(lda[bow_for_docs[loc]], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda.print_topic(i, 10)}\n")

Score: 0.9723467826843262	 
Topic: 0.015*"player" + 0.013*"season" + 0.009*"sport" + 0.008*"leagu" + 0.008*"coach" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball" + 0.004*"athlet" + 0.004*"match"

Score: 0.01307009533047676	 
Topic: 0.016*"student" + 0.008*"medic" + 0.008*"parent" + 0.007*"patient" + 0.007*"educ" + 0.007*"doctor" + 0.007*"colleg" + 0.006*"hospit" + 0.005*"test" + 0.005*"drug"



###### After viewing the original text and the cluster/classification is 'topic 1' which is sports with 97% confidence the model is correct. Then remaining 13% is placed on the 'topic 7' which is Education/Health.

### Testing LDA + TF-IDF model

In [193]:
for i, conf in sorted(lda_idf[bow_for_docs[loc]], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda_idf.print_topic(i, 10)}\n")

Score: 0.9860479831695557	 
Topic: 0.005*"season" + 0.005*"player" + 0.004*"leagu" + 0.004*"coach" + 0.003*"storm" + 0.003*"yanke" + 0.003*"hurrican" + 0.003*"water" + 0.002*"footbal" + 0.002*"basebal"



###### After viewing the original and the the cluster/classification. It appears that the model is 91% confident that the topic is 'topic 0' Which is sports. That is the correct classification for the document. The model is 5% and 2% sure that the document is 'topic 2' and 'topic 1' respectively.


### Test on new documents
article from: https://www.arabnews.com/node/1992296/sport

In [208]:
new_doc = 'It has been quite a year for football in Saudi Arabia, with the national team edging closer to the 2022 World Cup and Al-Hilal being crowned champions of Asia. Here is our look back at the best of 2021. Player of the Year: Salem Al-Dawsari Some years it is difficult to choose the player who has risen above the others, but in 2021 it is an easier choice than usual. Salem Al-Dawsari has been the best player in the Saudi Professional League with the team that was crowned champions. He was the best player in the team that won Asia’s biggest club prize, the AFC Champions League. He was the best player in the team that has been the best performing in qualification for the 2022 World Cup. If Asia was having an award this year, then Al-Dawsari would surely be pictured lifting the trophy in some swanky ceremony in an Asian capital. As it is, he will have to make do with being named as the MVP of the AFC Champions League and, of course, this Arab News accolade! The winger has not put a foot wrong all year and 2021 has been special with goals, assists and just all-round quality play at every level he has performed at. Goal of the Year: Nasser Al-Dawsari (vs Pohang Steelers, Nov. 23) There can’t have been a more influential goal in Asian club football for some time and it came in the biggest game of the year. This was the AFC Champions League final against Pohang Steelers in Riyadh, this was a game which Al-Hilal were desperate to win and become the first team in the history of Asian football to win four continental titles. The 22-year-old, in the team to replace Yasser Al-Shehrani on the left side of defence, picked up the ball in the middle of the Pohang half, advanced and then from around 25 meters unleashed an unstoppable shot that flew into the corner of the net. This came after just 16 seconds and it basically won the game with the Pohang coach admitting that it destroyed his game plan. It was a great goal that won a huge game. Coach of the Year: Herve Renard There are not many national teams anywhere that have improved as much in 2021 as Saudi Arabia. When it started, there was work to do to even be sure of a place in the final round of qualification for the 2022 World Cup. As the year ends, the Green Falcons are the form team in Asia and already have one foot in Qatar. The team have taken 16 points from six games, are four points above Japan in second and five above Australia in third. They will really have to mess up not to finish in the top two. It is not just about the results, the win over Japan or the disciplined draw in Sydney, but the way this increasingly tight unit is playing. Renard has introduced a level of intensity not seen for some time and the team is now as hard-working as they come. New players come in and know what they have to do and it was encouraging to see the Frenchman send an U-23 team to the Arab Cup to gain experience and competitive international action while he sat in the stands and watched. It was a far-sighted decision but, for now, the World Cup is close. Game of the Year: Al Fateh 5-5 Damac, Nov. 25 Just two days after Al-Hilal lifted the AFC Champions League trophy, there was a breathtaking game in the Saudi Pro League between two teams that don’t always make the headlines. In the previous 11 games, Al-Fateh had scored just 10 goals. In the previous 11, Damac had conceded the same number. There was little to suggest that this meeting would be the most thrilling of the year. There is not enough space to go into details of what happened on that Thursday as a team fighting relegation that had lost the previous three games traded goal after goal with the team in second. It was 2-2 after 19 minutes with Damac ahead on the stroke of half-time and then 4-2 up just after the hour. Then Al-Fateh hit three in seven minutes but the visitors still found time to make it 5-5. That it came from teams not part of the old elite and in the middle of the season meant that it was all the better. Team of the Year: Al-Hilal There is really not much of a debate with this one. When a team wins the domestic title and then goes and wins the biggest continental prize then it is hard to look elsewhere. Al-Hilal finished four points ahead of Al-Shabab with Bafetimbi Gomis scoring 24 times, but there is such strength in depth. The full-backs Yasser Al-Shahrani and Mohammed Al-Breik are one of the best pairs around. Then there is the skill and culture of Salman Al-Faraj and Salem Al-Dawsari and the energy and drive of Mohamed Kanno. Add strikers like Gomis and Moussa Marega and there is real firepower and added to the creativity of Matheus Pereira. It is a formidable team that can go through three coaches in 2021 and still keep winning. It has been a year to remember for Al-Hilal. Young Player of the Year: Nasser Al-Dawsari While there are high hopes over Al-Taawoun’s teenage defender Sulaiman Hazazi, Al-Dawsari, who turned 23 last week, has established himself as an important player for both club and country. This year, the versatile player — who can do a job in midfield or defence — has made over 30 appearances in the league for the champions. His performances were so impressive that he earned a call up to the national team from Renard. A debut came in a March friendly against Kuwait but a first competitive start came in the Green Falcons’ toughest game of the year: Australia in Sydney. It was a mature performance at left-back with the youngster coping well with the physical challenge presented by the Socceroos on their home turf. Throw in his crucial and spectacular goal in the final of the AFC Champions League and you have a spectacular season. Development of the Year: SAFF World Cup plans The proposal from the Saudi Arabia Football Federation that FIFA look into the possibility of holding the World Cup every two years instead of the current four has become a massive issue in football. It has divided the game with the reaction from Europe and South America largely negative, but with more support coming from Asia and elsewhere. It remains to be seen what will happen, but this idea that originated in Riyadh is going to run and run.'
bow = dict_of_words.doc2bow(preprocess(new_doc))

#### Testing LDA on new article

In [209]:
for i, conf in sorted(lda[bow], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda.print_topic(i, 10)}\n")

Score: 0.6370493173599243	 
Topic: 0.015*"player" + 0.013*"season" + 0.009*"sport" + 0.008*"leagu" + 0.008*"coach" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball" + 0.004*"athlet" + 0.004*"match"

Score: 0.3329722583293915	 
Topic: 0.008*"russia" + 0.008*"russian" + 0.008*"korea" + 0.007*"islam" + 0.007*"china" + 0.006*"syria" + 0.005*"minist" + 0.005*"nuclear" + 0.005*"muslim" + 0.005*"iran"

Score: 0.026898451149463654	 
Topic: 0.011*"reuter" + 0.009*"price" + 0.008*"bank" + 0.007*"stock" + 0.007*"insur" + 0.006*"delay" + 0.006*"economi" + 0.006*"worker" + 0.006*"thomson" + 0.005*"financ"



#### Result
The LDA model has classified the aritcle correctly into sports

#### Testing LDA + TF-IDF on new article

In [210]:
new_doc = 'It has been quite a year for football in Saudi Arabia, with the national team edging closer to the 2022 World Cup and Al-Hilal being crowned champions of Asia. Here is our look back at the best of 2021. Player of the Year: Salem Al-Dawsari Some years it is difficult to choose the player who has risen above the others, but in 2021 it is an easier choice than usual. Salem Al-Dawsari has been the best player in the Saudi Professional League with the team that was crowned champions. He was the best player in the team that won Asia’s biggest club prize, the AFC Champions League. He was the best player in the team that has been the best performing in qualification for the 2022 World Cup. If Asia was having an award this year, then Al-Dawsari would surely be pictured lifting the trophy in some swanky ceremony in an Asian capital. As it is, he will have to make do with being named as the MVP of the AFC Champions League and, of course, this Arab News accolade! The winger has not put a foot wrong all year and 2021 has been special with goals, assists and just all-round quality play at every level he has performed at. Goal of the Year: Nasser Al-Dawsari (vs Pohang Steelers, Nov. 23) There can’t have been a more influential goal in Asian club football for some time and it came in the biggest game of the year. This was the AFC Champions League final against Pohang Steelers in Riyadh, this was a game which Al-Hilal were desperate to win and become the first team in the history of Asian football to win four continental titles. The 22-year-old, in the team to replace Yasser Al-Shehrani on the left side of defence, picked up the ball in the middle of the Pohang half, advanced and then from around 25 meters unleashed an unstoppable shot that flew into the corner of the net. This came after just 16 seconds and it basically won the game with the Pohang coach admitting that it destroyed his game plan. It was a great goal that won a huge game. Coach of the Year: Herve Renard There are not many national teams anywhere that have improved as much in 2021 as Saudi Arabia. When it started, there was work to do to even be sure of a place in the final round of qualification for the 2022 World Cup. As the year ends, the Green Falcons are the form team in Asia and already have one foot in Qatar. The team have taken 16 points from six games, are four points above Japan in second and five above Australia in third. They will really have to mess up not to finish in the top two. It is not just about the results, the win over Japan or the disciplined draw in Sydney, but the way this increasingly tight unit is playing. Renard has introduced a level of intensity not seen for some time and the team is now as hard-working as they come. New players come in and know what they have to do and it was encouraging to see the Frenchman send an U-23 team to the Arab Cup to gain experience and competitive international action while he sat in the stands and watched. It was a far-sighted decision but, for now, the World Cup is close. Game of the Year: Al Fateh 5-5 Damac, Nov. 25 Just two days after Al-Hilal lifted the AFC Champions League trophy, there was a breathtaking game in the Saudi Pro League between two teams that don’t always make the headlines. In the previous 11 games, Al-Fateh had scored just 10 goals. In the previous 11, Damac had conceded the same number. There was little to suggest that this meeting would be the most thrilling of the year. There is not enough space to go into details of what happened on that Thursday as a team fighting relegation that had lost the previous three games traded goal after goal with the team in second. It was 2-2 after 19 minutes with Damac ahead on the stroke of half-time and then 4-2 up just after the hour. Then Al-Fateh hit three in seven minutes but the visitors still found time to make it 5-5. That it came from teams not part of the old elite and in the middle of the season meant that it was all the better. Team of the Year: Al-Hilal There is really not much of a debate with this one. When a team wins the domestic title and then goes and wins the biggest continental prize then it is hard to look elsewhere. Al-Hilal finished four points ahead of Al-Shabab with Bafetimbi Gomis scoring 24 times, but there is such strength in depth. The full-backs Yasser Al-Shahrani and Mohammed Al-Breik are one of the best pairs around. Then there is the skill and culture of Salman Al-Faraj and Salem Al-Dawsari and the energy and drive of Mohamed Kanno. Add strikers like Gomis and Moussa Marega and there is real firepower and added to the creativity of Matheus Pereira. It is a formidable team that can go through three coaches in 2021 and still keep winning. It has been a year to remember for Al-Hilal. Young Player of the Year: Nasser Al-Dawsari While there are high hopes over Al-Taawoun’s teenage defender Sulaiman Hazazi, Al-Dawsari, who turned 23 last week, has established himself as an important player for both club and country. This year, the versatile player — who can do a job in midfield or defence — has made over 30 appearances in the league for the champions. His performances were so impressive that he earned a call up to the national team from Renard. A debut came in a March friendly against Kuwait but a first competitive start came in the Green Falcons’ toughest game of the year: Australia in Sydney. It was a mature performance at left-back with the youngster coping well with the physical challenge presented by the Socceroos on their home turf. Throw in his crucial and spectacular goal in the final of the AFC Champions League and you have a spectacular season. Development of the Year: SAFF World Cup plans The proposal from the Saudi Arabia Football Federation that FIFA look into the possibility of holding the World Cup every two years instead of the current four has become a massive issue in football. It has divided the game with the reaction from Europe and South America largely negative, but with more support coming from Asia and elsewhere. It remains to be seen what will happen, but this idea that originated in Riyadh is going to run and run.'
bow = dict_of_words.doc2bow(preprocess(new_doc))

In [211]:
for i, conf in sorted(lda_idf[bow], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda_idf.print_topic(i, 10)}\n")

Score: 0.49581873416900635	 
Topic: 0.005*"season" + 0.005*"player" + 0.004*"leagu" + 0.004*"coach" + 0.003*"storm" + 0.003*"yanke" + 0.003*"hurrican" + 0.003*"water" + 0.002*"footbal" + 0.002*"basebal"

Score: 0.41033393144607544	 
Topic: 0.005*"reuter" + 0.003*"china" + 0.003*"korea" + 0.003*"thomson" + 0.002*"bank" + 0.002*"delay" + 0.002*"syria" + 0.002*"stock" + 0.002*"price" + 0.002*"nuclear"

Score: 0.09207019209861755	 
Topic: 0.002*"film" + 0.002*"music" + 0.001*"movi" + 0.001*"food" + 0.001*"artist" + 0.001*"song" + 0.001*"charact" + 0.001*"space" + 0.001*"season" + 0.001*"anim"



#### Result
The LDA+TF-IDF model has classified the aritcle correctly into sports